Cell 1 — Import required libraries

In [1]:
# Imports the 'os' module to interact with directories and file paths
import os

# Imports the ResNet50 model and its preprocessing function
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input

# Imports functions to load an image and convert it into a NumPy array
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Imports the Model class to customize the neural network
from tensorflow.keras.models import Model


Cell 2 — Load ResNet50 model

In [2]:
# Loads the pretrained ResNet50 model with default settings (ImageNet weights)
model = ResNet50()

# Restructures the model by removing the final classification layer
# and keeping outputs from the second last layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

# Displays the model summary to confirm the structure
print(model.summary())


102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

None


Cell 3 — Create a dictionary to store extracted features

In [3]:
# Creates an empty dictionary to hold the extracted feature vectors for each image
features = {}

Cell 4 — Set the directory containing image files

In [4]:
# Specifies the folder path that contains the images
directory = 'image data/'


Cell 5 — Loop through images, preprocess, and extract features

In [5]:
# Iterates through each file inside the specified directory
for image in os.listdir(directory):

    # Builds the full path for the current image file
    image_path = directory + image

    # Loads the image and resizes it to 224×224 (required by ResNet50)
    img = load_img(image_path, target_size=(224, 224))

    # Converts the loaded image into a NumPy array (pixel values)
    img = img_to_array(img)

    # Reshapes the image array to include a batch dimension: (1, height, width, channels)
    img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])

    # Applies ResNet50-specific preprocessing (mean subtraction, scaling, etc.)
    img = preprocess_input(img)

    # Extracts the feature vector by passing the image through the model
    feature = model.predict(img, verbose=0)

    # Stores the feature vector in the dictionary using the image path as the key
    features[image_path] = feature


In [6]:
features

{'image data/test.jpg': array([[0.33191985, 0.37943488, 0.01529721, ..., 0.12124984, 0.09195007,
         1.5163037 ]], dtype=float32)}